<a href="https://colab.research.google.com/github/vitaldb/examples/blob/master/ppf_bis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 프로포폴, 레미펜타닐 주입 속도로부터 BIS 예측

## VitalDB 데이터 셋 이용
본 예제에서는 오픈 생체 신호 데이터셋인 VitalDB를 이용하는 모든 사용자는 반드시 아래 Data Use Agreement에 동의하여야 합니다.

https://vitaldb.net/data-bank/?query=guide&documentId=13qqajnNZzkN7NZ9aXnaQ-47NWy7kx-a6gbrcEsi-gak&sectionId=h.usmoena3l4rb

동의하지 않을 경우 이 창을 닫으세요.

## 본 프로그램의 옵션들

In [ ]:
import numpy as np
import pandas as pd
import os

LSTM_TIMEPOINTS = 180
LSTM_NODES = 6
FNN_NODES = 16
BATCH_SIZE = 256
MAX_CASES = 10  # 본 예제에서 사용할 최대 case 수

# Data loading

## 트랙 데이터 가져오기


In [ ]:
df_trks = pd.read_csv('https://api.vitaldb.net/v2/trks')
ppf_cases = set(df_trks[df_trks['tname'] == 'Orchestra/PPF20_VOL']['caseid'])  # print(len(ppf_cases))  # 3523
rft_cases = set(df_trks[df_trks['tname'] == 'Orchestra/RFTN20_VOL']['caseid'])  # print(len(rft_cases))  # 4791
tiva_cases = ppf_cases & rft_cases  # print(len(tiva_cases))  # 3457
bis_cases = set(df_trks[df_trks['tname'] == 'BIS/BIS']['caseid'])  # print(len(bis_cases))  # 5867
study_cases = sorted(list(tiva_cases & bis_cases))  # print(len(study_cases))  # 3289

## 임상 데이터 가져오기 (나이, 성별, 키, 몸무게)
- clinical informaiton을 df_cases에 읽어오기
- 사용할 age, sex, weight, height 가져오기
- 성별이 F(=female)은 0으로, M(=male)은 1로 치환
- 18세 이상, 몸무게 35이상 기준으로 case 준비(inclusion, exclusion criteria)

In [ ]:
df_cases = pd.read_csv("https://api.vitaldb.net/cases")
cases = df_cases.loc[[caseid in study_cases for caseid in df_cases['caseid']], ['caseid', 'age', 'sex', 'weight', 'height']]
cases['sex'] = cases['sex'] == 'F'
cases = cases[cases['age'] > 18]  
cases = cases[cases['weight'] > 35]  
cases = cases[1:MAX_CASES]

## 1.2 임상 데이터 쪼개놓기
- 전체 case id 모으고(caseids)
- caseid별로 나이, 성별, 키, 몸무게 담기(caseid_aswh)
- 최대로 로딩 가능한 case 수 알기(ncase) 
- 그리고 caseid를 무작위로 섞음(.shuffle)
- dataset을 담을 변수를 설정(x_ppf, x_rrf, x_aswh, x_caseid, y)

In [ ]:
caseids = cases.values[:,0]
caseid_aswh = {row[0]: row[1:].astype(float) for row in cases.values}
ncase = min(MAX_CASES, len(caseids))
np.random.shuffle(caseids)

# vital 파일로부터 dataset 을 만듬
x_ppf = []  # 각 레코드의 프로포폴 주입량
x_rft = []  # 각 레코드의 레미펜타닐 주입량
x_aswh = []  # 각 레코드의 나이, 성별, 키, 몸무게
x_caseid = []  # 각 레코드의 caseid
y = []  # 각 레코드의 출력값 (bis)

## 1.3 각 트랙 로딩할 수 있는 함수 준비
- https://colab.research.google.com/github/vitaldb/pyvital/blob/master/vitaldb.ipynb?authuser=2#scrollTo=9SwIjOX9frk6 에서 다룬 [Numeric 트랙의 정규화], [여러 트랙을 한번에 로딩] 을 참고

In [ ]:
def load_trk(tid, interval=1):
    try:
        url = 'https://api.vitaldb.net/' + tid
        dtvals = pd.read_csv(url).values
    except:
        return np.empty(0)
    if len(dtvals) == 0:
        return np.empty(0)
    dtvals[:,0] /= interval  # convert time to row
    trklen = int(np.nanmax(dtvals[:,0])) + 1  # find maximum index (array length)
    ret = np.full(trklen, np.nan)  # create a dense array
    for idx, val in dtvals:  # copy values
        ret[int(idx)] = val
    return ret


def load_trks(tids, interval=1):
    trks = []
    maxlen = 0
    for tid in tids:
        trk = load_trk(tid, interval)
        trks.append(trk)
        if len(trk) > maxlen:
            maxlen = len(trk)
    if maxlen == 0:
        return np.empty(0)
    ret = np.full((maxlen, len(tids)), np.nan)  # create a dense array
    for i in range(len(tids)):  # copy values
        ret[:len(trks[i]), i] = trks[i]
    return ret
    

# DATA preparation

## 2.0 트랙 데이터(df_trks)에서 유효한 데이터 로딩
- propofol(ppf20), remifentanyl(rftn20), bis를 로딩한다
- 기록된 데이터가 짧거나, drug infusion이 실제 없었던 케이스, bis값이 적절하지 않은 케이스는 거르기
- 결측값 및 음수 처리

In [ ]:
PPF_DOSE = 0
RFT_DOSE = 1
BIS = 2
icase = 0

for caseid in caseids:
    ppf20_tid = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'Orchestra/PPF20_VOL')]['tid'].values[0]
    rftn20_tid = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'Orchestra/RFTN20_VOL')]['tid'].values[0]
    bis_tid = df_trks[(df_trks['caseid'] == caseid) & (df_trks['tname'] == 'BIS/BIS')]['tid'].values[0]
    vals = load_trks([ppf20_tid, rftn20_tid, bis_tid], 10) #10s 간격 추출

        # 2시간 이내의 case 들은 사용하지 않음, 720 =  2hr * 60min/hr * 6개/min
    if vals.shape[0] < 720:
        continue

    # 결측값은 측정된 마지막 값으로 대체
    vals = pd.DataFrame(vals).fillna(method='ffill').values
    vals = np.nan_to_num(vals, 0)  # 맨 앞 쪽 결측값은 0으로 대체

    # drug 주입을 하지 않은 경우 혹은 bis를 켜지 않은 경우 사용하지 않음
    if (np.max(vals, axis=0) <= 1).any():
        continue

    # drug infusion 시작 시간을 구하고 그 이전을 삭제
    first_ppf_idx = np.where(vals[:, PPF_DOSE] > 1)[0][0]
    first_rft_idx = np.where(vals[:, RFT_DOSE] > 1)[0][0]
    first_drug_idx = min(first_ppf_idx, first_rft_idx)
    vals = vals[first_drug_idx:, :]

    # volume 을 rate로 변경
    vals[1:, PPF_DOSE] -= vals[:-1, PPF_DOSE]
    vals[1:, RFT_DOSE] -= vals[:-1, RFT_DOSE]
    vals[0, PPF_DOSE] = 0
    vals[0, RFT_DOSE] = 0

    # 음수 값(volume 감소)을 0으로 대체
    vals[vals < 0] = 0

    # 유효한 bis 값들을 가져옴
    valid_bis_idx = np.where(vals[:, BIS] > 0)[0]

    # 2시간 이내의 case들은 사용하지 않음
    if valid_bis_idx.shape[0] < 720:
        continue

    # bis 값의 첫 값이 80 이하이거나 마지막 값이 70 이하인 case는 사용하지 않음
    first_bis_idx = valid_bis_idx[0]
    last_bis_idx = valid_bis_idx[-1]
    if vals[first_bis_idx, BIS] < 80 or vals[last_bis_idx, BIS] < 70:
        continue
    
    

    icase += 1
    print('loading ({}/{}): caseid {:.0f}, first bis {:.1f}, last bis {:.1f}'.format(icase, ncase, caseid, vals[first_bis_idx, BIS], vals[last_bis_idx, BIS]))

    # infusion 시작 전 LSTM_TIMEPOINTS 동안의 dose와 bis를 모두 0으로 세팅
    vals = np.vstack((np.zeros((LSTM_TIMEPOINTS - 1, 3)), vals))



    # 현 case의 나이, 성별, 키, 몸무게를 가져옴
    aswh = caseid_aswh[caseid]

    # case 시작 부터 종료 까지 dataset 에 넣음
    for irow in range(1, vals.shape[0] - LSTM_TIMEPOINTS - 1):
        bis = vals[irow + LSTM_TIMEPOINTS, BIS]
        if bis == 0:
            continue

        # 데이터셋에 입력값을 넣음
        x_ppf.append(vals[irow:irow + LSTM_TIMEPOINTS, PPF_DOSE])
        x_rft.append(vals[irow:irow + LSTM_TIMEPOINTS, RFT_DOSE])
        x_aswh.append(aswh)
        x_caseid.append(caseid)
        y.append(bis)

    if icase >= ncase:
        break
    

## 2.1 확인된 데이터 담기

In [ ]:
# 모든 케이스 확인, xppf... y까지 담은후,
# 입력 데이터셋을 numpy array로 변경

x_ppf = np.array(x_ppf)[..., None]  # LSTM 에 넣기 위해서는 3차원이어야 한다. 마지막 차원을 추가
x_rft = np.array(x_rft)[..., None]
x_aswh = np.array(x_aswh)
y = np.array(y)
x_caseid = np.array(x_caseid)

# 최종적으로 로딩 된 caseid
caseids = np.unique(x_caseid)

# normalize data
x_aswh = (x_aswh - np.mean(x_aswh, axis=0)) / np.std(x_aswh, axis=0)

# bis 값은 최대값이 98 이므로 98로 나눠 normalization
y /= 98


## 2.2 데이터를 학습(train)과 테스트(test)로 나누기

In [ ]:
# train, test case로 나눔
ntest = int(ncase * 0.1)
nval = int(ncase * 0.1)
ntrain = ncase - nval - ntest
train_caseids = caseids[:ntrain]
val_caseids = caseids[ntrain:ntrain + nval]
test_caseids = caseids[ncase - ntest:ncase]

# train set과 test set 으로 나눔
train_mask = np.array([caseid in train_caseids for caseid in x_caseid])
val_mask = np.array([caseid in val_caseids for caseid in x_caseid])
test_mask = np.array([caseid in test_caseids for caseid in x_caseid])
x_train = [x_ppf[train_mask], x_rft[train_mask], x_aswh[train_mask]]
y_train = y[train_mask]
x_val = [x_ppf[val_mask], x_rft[val_mask], x_aswh[val_mask]]
y_val = y[val_mask]
x_test = [x_ppf[test_mask], x_rft[test_mask], x_aswh[test_mask]]
y_test = y[test_mask]

print('train: {} cases {} samples'.format(len(train_caseids), np.sum(train_mask)))
print('val: {} cases {} samples'.format(len(val_caseids), np.sum(val_mask)))
print('test: {} cases {} samples'.format(len(test_caseids), np.sum(test_mask)))

# Model building


## 3.1 필요한 module과 output폴더

In [ ]:
from keras.models import Model, load_model
from keras.layers import Dense, Dropout, LSTM, Input, concatenate
from keras.callbacks import ModelCheckpoint, EarlyStopping

# 출력 폴더를 생성
odir = 'output'
if not os.path.exists(odir):
    os.mkdir(odir)
weight_path = odir + "/weights.hdf5"

## 3.2 학습마다 데이터를 넘겨줄 함수

In [ ]:
def gen():
    global BATCH_SIZE, x_ppf, x_rft, x_aswh, y, train_mask
    train_idx = np.where(train_mask)[0]
    while True:
        batch_idx = np.random.choice(train_idx, BATCH_SIZE)
        yield [np.take(x_ppf, batch_idx, axis=0), np.take(x_rft, batch_idx, axis=0), np.take(x_aswh, batch_idx, axis=0)], [np.take(y, batch_idx, axis=0)]

## 3.3 모델 설계


In [ ]:
input_cov = Input(batch_shape=(None, 4))
input_ppf = Input(batch_shape=(None, LSTM_TIMEPOINTS, 1))
input_rft = Input(batch_shape=(None, LSTM_TIMEPOINTS, 1))
output_ppf = LSTM(LSTM_NODES, input_shape=(LSTM_TIMEPOINTS, 1), activation='tanh')(input_ppf)
output_rft = LSTM(LSTM_NODES, input_shape=(LSTM_TIMEPOINTS, 1), activation='tanh')(input_rft)
output = concatenate([output_ppf, output_rft, input_cov])
output = Dense(FNN_NODES, activation='relu')(output)
output = Dropout(0.2)(output)
output = Dense(1, activation='sigmoid')(output)


## 3.4 모델 학습

In [ ]:
model = Model(inputs=[input_ppf, input_rft, input_cov], outputs=[output])
model.compile(loss='MSE', optimizer='adam')
hist = model.fit_generator(gen(), validation_data=[x_val, y_val], epochs=100, steps_per_epoch=100,
                           callbacks=[ModelCheckpoint(monitor='val_loss', filepath=weight_path, verbose=1, save_best_only=True),
                                      EarlyStopping(monitor='val_loss', patience=0, verbose=0, mode='auto')])
# model.load_weights(weight_path)

## 모델 성능 평가 및 그림 출력

In [ ]:
y_pred = model.predict(x_test).flatten()
test_err = y_pred - y_test
test_rmse = np.mean(np.square(test_err)) ** 0.5

print("Test RMSE: {}".format(test_rmse))

# 각 case에서 예측 결과를 그림으로 확인
for caseid in test_caseids:
    case_mask = (x_caseid[test_mask] == caseid)
    case_len = np.sum(case_mask)
    if case_len == 0:
        continue

    case_rmse = np.mean(np.square(test_err[case_mask])) ** 0.5
    print('{}\t{}'.format(caseid, case_rmse))

import matplotlib.pyplot as plt
    t = np.arange(0, case_len)
    plt.figure(figsize=(20, 5))
    plt.plot(t, y_test[case_mask], t, y_pred[case_mask])
    plt.xlim([0, case_len])
    plt.ylim([0, 1])
    plt.savefig('{}/{:.3f}_{}.png'.format(odir, case_rmse, caseid))
    plt.close()